In [2]:
!pip install transformers==4.49.0 pyspark

  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 77.9 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0


In [2]:
!pip install sentencepiece

In [3]:
from transformers import BartTokenizer, BartForConditionalGeneration, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoProcessor
import torch

import torch.nn as nn

import pandas as pd

In [4]:
# model_id = "google/flan-t5-small"
model_id="facebook/bart-base"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = BartTokenizer.from_pretrained(model_id)
model = BartForConditionalGeneration.from_pretrained(model_id)
model = model.to(device)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
processor = AutoProcessor.from_pretrained(model_id)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [5]:
# import torch
# import torch.nn as nn

# if torch.cuda.device_count() > 1:
#     print("Let's use", torch.cuda.device_count(), "GPUs!")
#     model = nn.DataParallel(model)

# model = model.to("cuda")


In [6]:
print(device)

cuda


In [8]:
# for param in model.encoder.parameters():
#     param.requires_grad = False  # Freeze encoder

# for param in model.decoder.parameters():
#     param.requires_grad = False  # Freeze decoder

# # Only train the final output layer
# for param in model.lm_head.parameters():
#     param.requires_grad = True

In [9]:
!pip install datasets

In [10]:
import pandas as pd
url = "https://raw.githubusercontent.com/Hawawou/DS_QA_bot/refs/heads/main/data/data1.csv"


data = pd.read_csv(url)
data.drop(columns=["Unnamed: 0"], inplace=True)
data.head()

,question,answer
0,What is Natural Language Processing (NLP)?,Natural Language Processing is a field of arti...
1,How does NLP relate to Computer Vision?,NLP and Computer Vision often complement each ...
2,What is the role of tokenization in NLP?,Tokenization is the process of breaking down t...
3,Can you explain the concept of word embeddings?,Word embeddings are numerical representations ...
4,What is object recognition in Computer Vision?,Object recognition is a Computer Vision task t...


In [11]:
from datasets import Dataset

dataset = Dataset.from_pandas(data)
dataset = dataset.train_test_split(test_size=0.2)
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 9357
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 2340
    })
})

In [12]:
data_train = dataset["train"]
data_test = dataset["test"]

In [13]:
assistant = "You are a datascience tutor. Users will ask you you data science questions and you will provide them with the answers."

import ast  # To safely evaluate the string list representation

def format_data(examples):
    inputs = [assistant + " " + doc for doc in examples["question"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    labels = tokenizer(text_target = examples["answer"], max_length=512, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs



In [14]:
tokenized_dataset = data_train.map(format_data, batched=True)
tokenized_dataset

Map:   0%|          | 0/9357 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 9357
})

In [15]:
data_test = data_test.map(format_data, batched=True)

Map:   0%|          | 0/2340 [00:00<?, ? examples/s]

In [16]:
print(tokenized_dataset)

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 9357
})


In [17]:
# print(tokenized_dataset['input_ids'])

In [18]:
!pip install nltk rouge_score evaluate

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.7 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=307bb90eae2e8e33539660f4e0b82d771f35defd3a2a4a32ce1f46b4dd726269
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [19]:
# Evaluation
import evaluate
import nltk
import numpy as np


nltk.download('punkt')
metric = evaluate.load("rouge")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
def compute_metrics(eval_preds):
   preds, labels = eval_preds

   # decode preds and labels
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

   # rougeLSum expects newline after each sentence
   decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
   decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

   result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  
   return result

In [21]:
# print('Preds shape:' pre)

In [22]:
# parameters
learning_rate = 1e-5
batch_size = 16
per_device_train_batch_size = 8
per_device_eval_batch_size = 8
weight_decay = 0.01
num_epochs = 5

In [23]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    push_to_hub=True,
    hub_model_id='Hawat/DS-QA',
    hub_strategy="every_save",
    eval_strategy="epoch",
    fp16=True,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=weight_decay,
    num_train_epochs=num_epochs,
    predict_with_generate=True,
    logging_dir="./logs",
    report_to="none",
    # generation_max_length =64,
    # dataloader_num_workers=4
)

In [25]:
trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset,
   eval_dataset=data_test,
   processing_class = processor,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)

In [26]:
!rm -rf /kaggle/working/results/checkpoint-*
# !rm -rf /kaggle/working/logs


In [27]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,4.065800,3.512259,0.147024,0.036593,0.114381,0.127447
2,3.774500,3.435309,0.145507,0.038519,0.112340,0.126441
3,3.648200,3.397095,0.151653,0.042500,0.118884,0.131779
4,3.592100,3.377118,0.157388,0.044762,0.123738,0.136314
5,3.564100,3.372135,0.159001,0.045203,0.125393,0.138359


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=2925, training_loss=3.702486770499466, metrics={'train_runtime': 3343.3421, 'train_samples_per_second': 13.993, 'train_steps_per_second': 0.875, 'total_flos': 1.069198305057792e+16, 'train_loss': 3.702486770499466, 'epoch': 5.0})

In [30]:
trainer.save_model("./chat")

No files have been modified since last commit. Skipping to prevent empty commit.


In [44]:
last_checkpoint = "/kaggle/working/results/checkpoint-1000"

finetuned_model = BartForConditionalGeneration.from_pretrained(last_checkpoint)
tokenizer = BartTokenizer.from_pretrained(last_checkpoint)

In [40]:
my_question = "What is reinforcement learning?"
inputs = "You are a datascience tutor. Users will ask you you data science questions and you will provide them with the answers. Answer the following question: " + my_question
print(inputs)

You are a datascience tutor. Users will ask you you data science questions and you will provide them with the answers. Answer the following question: What is reinforcement learning?


In [41]:
inputs = tokenizer(inputs, return_tensors="pt")
outputs = finetuned_model.generate(**inputs)
answer = tokenizer.decode(outputs[0])
print(answer)

</s><s>Reinforcement learning is a technique used in machine learning to train models to improve the performance</s>


In [42]:
!zip -r model.zip chat/

  adding: chat/ (stored 0%)
  adding: chat/tokenizer.json (deflated 82%)
  adding: chat/config.json (deflated 63%)
  adding: chat/training_args.bin (deflated 52%)
  adding: chat/vocab.json (deflated 59%)
  adding: chat/generation_config.json (deflated 45%)
  adding: chat/special_tokens_map.json (deflated 52%)
  adding: chat/merges.txt (deflated 53%)
  adding: chat/tokenizer_config.json (deflated 75%)
  adding: chat/model.safetensors (deflated 7%)


In [43]:
!mv /kaggle/temp/model.zip /kaggle/working/


mv: cannot stat '/kaggle/temp/model.zip': No such file or directory
